In [1]:
!pip install --upgrade pip
!pip install python-binance
!pip install data_cache

# Option valuation model based on historical prices

In [72]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'
    
os.environ['CACHE_PATH'] = path
os.environ['DISABLE_CACHE'] = 'FALSE'

from datetime import datetime, timedelta, date
import pandas
import binance
import requests
from datetime import timezone
from typing import Tuple

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'


def first_day_of_next_month(year: int, month: int) -> datetime:
    """Returns the first datetime of the next month.
    
    Args:
    year: The year.
    month: The month.
    
    Returns:
    A datetime object representing the first day of the next month.
    """
    
    next_month = month + 1
    if next_month > 12:
        next_month = 1
        year += 1
    return datetime(year, next_month, 1)


def load_prices_by_month(code: str, year: int, month: int, force_refresh: bool=False) -> pandas.DataFrame:
    target_path = f'{path}/{BINANCE_SYMBOL}/{year}'
    target_filename = f'{target_path}/{year}-{month:02d}.csv.zip'
    if os.path.exists(target_filename) and not force_refresh:
        binance_prices = pandas.read_csv(target_filename, compression='infer', header=0)
    else:
        print(f'no previous data found in {target_path}, loading from binance')
        binance_client = binance.Client()
        from_date = datetime(year, month, 1, 0, 0, 0)
        until_date = first_day_of_next_month(year, month) - timedelta(seconds=1)
        
        candles = binance_client.get_historical_klines(code, binance.Client.KLINE_INTERVAL_1HOUR, str(from_date), str(until_date))
        binance_prices = pandas.DataFrame(candles, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
                                              'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol',
                                              'ignore'])
        os.makedirs(target_path, exist_ok=True)
        binance_prices.to_csv(target_filename, index=False, compression="zip")
    
    # as timestamp is returned in ms, let us convert this back to proper timestamps.
    binance_prices['open'] = binance_prices['open'].astype(float)
    binance_prices['high'] = binance_prices['high'].astype(float)
    binance_prices['low'] = binance_prices['low'].astype(float)
    binance_prices['close'] = binance_prices['close'].astype(float)
    binance_prices['volume'] = binance_prices['volume'].astype(float)
    binance_prices.dateTime = pandas.to_datetime(binance_prices.dateTime, unit='ms')
    binance_prices.set_index('dateTime', inplace=True)
    return binance_prices

current_year = datetime.now().year
df_by_period = list()
for year in range(current_year - COUNT_YEARS, current_year + 1):
    print(f'\nloading {year}', end=' ')
    for month in range(1, 13):
        if year == current_year and month == datetime.today().month:
            print(f'\ninterrupting at {year}/{month:02d}')
            break
        print('.', end='')
        df = load_prices_by_month(BINANCE_SYMBOL, year, month)
        df = df.drop(['closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'], axis=1)
        df_by_period.append(df)

prices_df = pandas.concat(df_by_period, axis=0)
prices_df.index = pandas.to_datetime(prices_df.index)

open_prices = prices_df['open']


loading 2014 ............
loading 2015 ............
loading 2016 ............
loading 2017 ............
loading 2018 ............
loading 2019 ............
loading 2020 ............
loading 2021 ............
loading 2022 ............
loading 2023 ............
loading 2024 
interrupting at 2024/01


## Option valuation model: input parameters here

In [94]:
target_period_hours = 2 * 24 + 12
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 7)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = 'https://www.deribit.com/api/v2/public'

get_options = f"{base_url}/get_instruments?currency={instrument_code}&kind=option&expired=false"
response_options = requests.get(get_options, headers=headers)
if response_options.status_code != 200:
    raise IOError(f'request failed with error {response_options.status_code}')

result = response_options.json()['result']
puts = {}
calls = {}
for option in result:
    if option['option_type'] == 'put':
        puts[(option['strike'], datetime.fromtimestamp(option['expiration_timestamp'] / 1000))] = option['instrument_id']
    elif option['option_type'] == 'call':
        calls[(option['strike'], datetime.fromtimestamp(option['expiration_timestamp'] / 1000))] = option['instrument_id']

target_expiry = min({k[1] for k in puts.keys()}, key=lambda d: abs(d - (datetime.now() + timedelta(hours=target_period_hours))))
strikes = {strike for strike, _ in puts.keys()}

get_current_price = f"{base_url}/get_index_price?index_name={instrument_code.lower()}_usd"
response_current_price = requests.get(get_current_price, headers=headers)

if response_current_price.status_code != 200:
    raise IOError(f'request failed with error {response_current_price.status_code}')

current_price = response_current_price.json()['result']['index_price']
remaining_hours = int((target_expiry - datetime.now()).total_seconds() / 3600)

period_close_series = prices_df['close'].shift(-remaining_hours)
df = pandas.DataFrame({
    'prices': open_prices,
    'period_close_series': period_close_series,
})
df.dropna(inplace=True)
df = df.loc[map(lambda ind: (ind.year, ind.month) >= CUT_OFF_YEAR_MONTH, df.index)]


def generate_strikes(price: float, option_strikes, count_options):
    sorted_strikes = sorted(option_strikes)
    closest_strike = min(sorted_strikes, key=lambda s: abs(s - price))
    closest_strike_pos = sorted_strikes.index(closest_strike)
    return sorted_strikes[closest_strike_pos - count_options : closest_strike_pos + count_options + 1]

def load_bid_ask(options, strike: float, expiry: date) -> Tuple[float, float]:
    instrument_id = options[(strike, expiry)]
    get_bid_ask = f"{base_url}/get_order_book_by_instrument_id?instrument_id={instrument_id}&depth=1"
    response_bid_ask = requests.get(get_bid_ask, headers=headers)

    if response_bid_ask.status_code != 200:
        raise IOError(f'request failed with error {response_bid_ask.status_code}')

    bid_ask = response_bid_ask.json()['result']
    bid = ask = None
    if 'bids' in bid_ask and bid_ask['bids'] and bid_ask['bids'][0] and bid_ask['bids'][0][0]:
        bid = bid_ask['bids'][0][0]
    if 'asks' in bid_ask and bid_ask['asks'] and bid_ask['asks'][0] and bid_ask['asks'][0][0]:
        ask = bid_ask['asks'][0][0]
    return bid, ask

strike_prices = generate_strikes(current_price, strikes, strikes_universe_size)

for count, strike_price in enumerate(strike_prices, start=1):
    strike_factor = strike_price / current_price
    df[f'strike_{count}'] = df['prices'].multiply(strike_factor)
    df[f'strike_pct_{count}'] = strike_factor

    df[f'call_value_{count}'] = df['period_close_series'].subtract(df[f'strike_{count}'])
    df.loc[df[f'call_value_{count}'] < 0., f'call_value_{count}'] = 0.
    df[f'call_value_pct_{count}'] = df[f'call_value_{count}'].divide(df['prices'])

    df[f'put_value_{count}'] = df[f'strike_{count}'].subtract(df['period_close_series'])
    df.loc[df[f'put_value_{count}'] < 0., f'put_value_{count}'] = 0.
    df[f'put_value_pct_{count}'] = df[f'put_value_{count}'].divide(df['prices'])

option_chain = list()
for count, strike_price in enumerate(strike_prices, start=1):
    put_bid, put_ask = load_bid_ask(puts, strike_price, target_expiry)
    call_bid, call_ask = load_bid_ask(calls, strike_price, target_expiry)
    strike_data = {
        'strike': strike_price,
        'value_call': df[f'call_value_pct_{count}'].mean() * current_price,
        'value_put': df[f'put_value_pct_{count}'].mean() * current_price,
        'value_call_median': df[f'call_value_pct_{count}'].quantile(0.5) * current_price,
        'value_put_median': df[f'put_value_pct_{count}'].quantile(0.5) * current_price,
        'value_call_pct': df[f'call_value_pct_{count}'].mean(),
        'call_ask': call_ask,
        'value_put_pct': df[f'put_value_pct_{count}'].mean(),
        'put_ask': put_ask
    }
    if strike_price < current_price:
        strike_data['value_call_pct'] = None
    if strike_price > current_price:
        strike_data['value_put_pct'] = None

    option_chain.append(strike_data)

option_chain_df = pandas.DataFrame(option_chain).set_index('strike').sort_index()

#
index_put = strikes_universe_size - 4
index_call = strikes_universe_size + 4
#
cost, value = (option_chain_df.iloc[index_put]['put_ask'] + option_chain_df.iloc[index_call]['call_ask'], 
 option_chain_df.iloc[index_put]['value_put_pct'] + option_chain_df.iloc[index_call]['value_call_pct']
)
print(f'target expiry: {target_expiry.astimezone(timezone.utc)}')
print(f'trading put {option_chain_df.iloc[index_put].name:.0f} and call {option_chain_df.iloc[index_call].name:.0f}')
print(f'cost: {cost:.3f} / value: {value:.3f}, benefit/cost = {value / cost:.1f}x')
print(f'($) cost: {cost * current_price:.2f} / value: {value * current_price:.2f}, average gain = {(value - cost) * current_price:.2f}')

option_chain_df

target expiry: 2024-01-28 08:00:00+00:00
trading put 2100 and call 2300
cost: 0.005 / value: 0.015, benefit/cost = 2.8x
($) cost: 11.61 / value: 32.40, average gain = 20.79


,value_call,value_put,value_call_median,value_put_median,value_call_pct,call_ask,value_put_pct,put_ask
strike,,,,,,,,
2100.0,112.186924,17.016764,92.509527,0.000000,NaN,0.0520,0.007768,0.0034
2125.0,92.236952,22.066793,67.509527,0.000000,NaN,0.0420,0.010073,0.0049
2150.0,73.934707,28.764548,42.509527,0.000000,NaN,0.0340,0.013131,0.0075
2175.0,57.907276,37.737117,17.509527,0.000000,NaN,0.0190,0.017227,0.0105
2200.0,44.748559,49.578400,0.000000,7.490473,0.020427,0.0125,NaN,0.0155
2225.0,34.499612,64.329453,0.000000,32.490473,0.015749,0.0080,NaN,0.0250
2250.0,26.457036,81.286877,0.000000,57.490473,0.012077,0.0046,NaN,NaN
2275.0,20.197891,100.027732,0.000000,82.490473,0.009220,0.0030,NaN,NaN
2300.0,15.386279,120.216120,0.000000,107.490473,0.007024,0.0019,NaN,0.0665


In [86]:
df.head()

,prices,period_close_series,strike_1,strike_pct_1,call_value_1,call_value_pct_1,put_value_1,put_value_pct_1,strike_2,strike_pct_2,...,call_value_8,call_value_pct_8,put_value_8,put_value_pct_8,strike_9,strike_pct_9,call_value_9,call_value_pct_9,put_value_9,put_value_pct_9
dateTime,,,,,,,,,,,,,,,,,,,,,
2021-07-01 00:00:00,2275.68,2112.36,2165.477803,0.951574,0.000000,0.000000,53.117803,0.023342,2191.257301,0.962902,...,0.0,0.0,233.574287,0.102639,2371.713785,1.0422,0.0,0.0,259.353785,0.113968
2021-07-01 01:00:00,2245.32,2109.55,2136.588018,0.951574,0.000000,0.000000,27.038018,0.012042,2162.023590,0.962902,...,0.0,0.0,205.087020,0.091340,2340.072591,1.0422,0.0,0.0,230.522591,0.102668
2021-07-01 02:00:00,2260.58,2093.04,2151.109037,0.951574,0.000000,0.000000,58.069037,0.025688,2176.717478,0.962902,...,0.0,0.0,237.328123,0.104986,2355.976564,1.0422,0.0,0.0,262.936564,0.116314
2021-07-01 03:00:00,2197.73,2103.73,2091.302614,0.951574,12.427386,0.005655,0.000000,0.000000,2116.199074,0.962902,...,0.0,0.0,161.847832,0.073643,2290.474292,1.0422,0.0,0.0,186.744292,0.084971
2021-07-01 04:00:00,2184.79,2090.68,2078.989247,0.951574,11.690753,0.005351,0.000000,0.000000,2103.739119,0.962902,...,0.0,0.0,161.558351,0.073947,2276.988223,1.0422,0.0,0.0,186.308223,0.085275
